# GenomeVedic Terra.bio Quick Start

This notebook demonstrates how to use GenomeVedic in Terra.bio for interactive genome visualization.

**Requirements:**
- Terra workspace with GCS access
- BAM file in workspace bucket
- GenomeVedic API running (local or remote)

**Time:** ~5 minutes

## 1. Installation

Install GenomeVedic with Terra support:

In [ ]:
# Install GenomeVedic (run once per notebook session)
!pip install genomevedic[terra] -q

print("✓ Installation complete!")

## 2. Import and Verify

In [ ]:
import genomevedic as gv
import os

print(f"GenomeVedic version: {gv.__version__}")
print(f"Terra workspace: {os.getenv('WORKSPACE_NAMESPACE', 'Not detected')}")
print(f"GCS project: {os.getenv('GOOGLE_PROJECT', 'Not detected')}")

## 3. Quick Start: One-Line Visualization

The simplest way to visualize a BAM file:

In [ ]:
# Replace with your BAM file path
BAM_FILE = "gs://your-workspace-bucket/sample.bam"

# One line to visualize!
gv.show(bam_file=BAM_FILE)

## 4. Advanced: Widget with Options

In [ ]:
# Create widget with specific region (BRCA1 gene)
widget = gv.GenomeVedicWidget(
    bam_file=BAM_FILE,
    reference="hg38",
    initial_region="chr17:41196311-41277500",  # BRCA1
    height="600px"
)

widget.show()

## 5. Natural Language Queries

Query genomic data using plain English:

In [ ]:
# Query for variants
result = widget.query("Find all pathogenic variants in BRCA1")

print(f"Original query: {result['original_query']}")
print(f"Generated SQL: {result['generated_sql']}")
print(f"Explanation: {result['explanation']}")

In [ ]:
# Try different queries
queries = [
    "Find SNPs on chromosome 17",
    "Show mutations with MAF > 0.01",
    "List variants in exon regions"
]

for query in queries:
    result = widget.query(query)
    print(f"\n{query}")
    print(f"  → {result['generated_sql'][:100]}...")

## 6. AI Variant Explanations

In [ ]:
# Get explanation for a specific variant
explanation = widget.explain_variant(
    gene="BRCA1",
    variant="c.68_69delAG",
    cancer_type="breast cancer"
)

print(f"Gene: {explanation.get('gene')}")
print(f"Variant: {explanation.get('variant')}")
print(f"\nSummary: {explanation.get('summary')}")
print(f"\nClinical Significance: {explanation.get('clinical_significance')}")

## 7. Working with GCS Files

In [ ]:
# Check file access
if gv.check_gcs_access(BAM_FILE):
    print("✓ File is accessible")
else:
    print("✗ Cannot access file - check permissions")

# Get file information
from genomevedic import TerraGCSClient

client = TerraGCSClient()
info = client.get_file_info(BAM_FILE)

print(f"\nFile: {info['name']}")
print(f"Size: {info['size']:,} bytes ({info['size']/1e9:.2f} GB)")
print(f"Bucket: {info['bucket']}")
print(f"Created: {info['created']}")

## 8. Compare Multiple Samples

In [ ]:
# Compare tumor vs normal
gv.create_comparison_view(
    bam_files=[
        "gs://bucket/tumor.bam",
        "gs://bucket/normal.bam"
    ],
    labels=["Tumor", "Normal"],
    reference="hg38",
    initial_region="chr17:7571719-7590868"  # TP53
)

## 9. Configuration

In [ ]:
# Configure GenomeVedic settings
gv.set_api_url("http://localhost:8080")  # Your GenomeVedic instance
gv.set_default_reference("hg38")

# View current configuration
config = gv.get_config()
print("Current configuration:")
for key, value in config.items():
    print(f"  {key}: {value}")

## 10. Share Results

In [ ]:
# Get shareable URL
url = widget.get_session_url()
print(f"Share this URL: {url}")

# Generate temporary signed URL for GCS file
from genomevedic import GCSClient

client = GCSClient()
signed_url = client.generate_signed_url(BAM_FILE, expiration=3600)
print(f"\nTemporary file URL (1 hour): {signed_url[:100]}...")

## Next Steps

- See `terra_tcga_analysis.ipynb` for TCGA dataset example
- See `advanced_features.ipynb` for more advanced usage
- Read full docs: https://genomevedic.readthedocs.io